# Setup environment

In [2]:
import os
import json

with open(".env.json", "r") as f_in:
    env_json: dict = json.load(f_in)
    for key, value in env_json.items():
        os.environ[key] = value

# Helper Functions

In [3]:
import pandas as pd
import numpy as np

In [4]:
path_data = "C:/Users/usuario/Documents/GitHub/101MachineLearning/000_data"

# Data

In [5]:
column_name_content_ciiu = "CIIU4_DESC"

df_ciiu = pd.read_excel(f"{path_data}/CIIU.xlsx", sheet_name="CIIU 4.0")
df_ciiu.info()

df_ciiu_4 = df_ciiu[["CIIU4_COD", "CIIU4_DESC", "CIIU4_NIVEL"]]
print("CIIU 4")
print(df_ciiu_4.count())
df_ciiu_4.head()

df_ciiu_4_level2 = df_ciiu_4[df_ciiu_4["CIIU4_NIVEL"] == 2].drop_duplicates(["CIIU4_COD"]).reset_index()
print("CIIU 4 - Level 2")
print(df_ciiu_4_level2.count())
df_ciiu_4_level2.head(10)

df_ciiu_4_level6 = df_ciiu_4[df_ciiu_4["CIIU4_NIVEL"] == 6].drop_duplicates(["CIIU4_COD"]).reset_index()
print("CIIU 4 - Level 6")
print(df_ciiu_4_level6.count())
df_ciiu_4_level6.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6919 entries, 0 to 6918
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   CIIU4_COD    6919 non-null   object
 1   CIIU4_DESC   6919 non-null   object
 2   CIIU4_NIVEL  6919 non-null   object
 3   CIIU3_COD    4788 non-null   object
 4   CIIU3_DESC   4788 non-null   object
 5   CIIU3_NIVEL  5852 non-null   object
dtypes: object(6)
memory usage: 324.5+ KB
CIIU 4
CIIU4_COD      6919
CIIU4_DESC     6919
CIIU4_NIVEL    6919
dtype: int64
CIIU 4 - Level 2
index          91
CIIU4_COD      91
CIIU4_DESC     91
CIIU4_NIVEL    91
dtype: int64
CIIU 4 - Level 6
index          1908
CIIU4_COD      1908
CIIU4_DESC     1908
CIIU4_NIVEL    1908
dtype: int64


,index,CIIU4_COD,CIIU4_DESC,CIIU4_NIVEL
0,6,A0111.11,Cultivo de trigo.,6
1,8,A0111.12,Cultivo de maíz.,6
2,10,A0111.13,Cultivo de quinua.,6
3,13,A0111.19,"Otros cultivos de cereales n.c.p.: sorgo, ceba...",6
4,16,A0111.21,Cultivo de fréjol.,6


In [15]:
# importing all the required modules
import PyPDF2

# creating a pdf reader object
file_name_0 = f"{path_data}/ley_organica_de_proteccion_de_datos_personales.pdf"
reader = PyPDF2.PdfReader(file_name_0)

print(len(reader.pages))
pages_text_only = [reader.pages[i].extract_text() for i in range(len(reader.pages))]
pages_text_only_full = " ".join(pages_text_only)
print(pages_text_only_full[0:100], len(pages_text_only_full))

38
LEY ORGÁNICA DE PROTECCIÓN DE DATOS
PERSONALES
Ley 0
Registro Oficial Suplemento 459 de 26-may.-2021 141871


# Embeddings

In [106]:
# from .Embedder import Embedder, EmbedderAWS

import tiktoken
from langchain.text_splitter import RecursiveCharacterTextSplitter
import PyPDF2
import pandas as pd

class Embedder:

    def __init__(self, configuration, encoder):
        self.config_model = configuration["model"]
        self.config_splitter = configuration["splitter"]
        self.encoder = encoder
        self._text_splitter = None

    def get_splitter(self):

        if self._text_splitter is None:

            self._text_splitter = RecursiveCharacterTextSplitter(
                chunk_size=self.config_splitter["chunk_size"],
                chunk_overlap=self.config_splitter["chunk_overlap"],
                length_function=self.config_splitter["length_function"],
                add_start_index=self.config_splitter["add_start_index"],
            )
        return self._text_splitter
    
    def split_and_explode(self, df_to_split_explode, column_name_content_ciiu):
        # TODO: optimizar copia que se haga en el mismo dataframe
        column_name_content_ciiu_split = column_name_content_ciiu + "_split"
        # TODO: Paralelizable
        df_to_split_explode[column_name_content_ciiu_split] = df_to_split_explode[column_name_content_ciiu].apply(self.get_splitter().split_text)
        # TODO: Paralelizable
        return df_to_split_explode.drop([column_name_content_ciiu], axis=1).explode(column_name_content_ciiu_split).rename(columns={column_name_content_ciiu_split: column_name_content_ciiu})
            
    def embed_dataframe(self, df_to_embed, column_name_content_ciiu, check_cost=True):

        if check_cost:
            total_cost, total_tokens = Embedder.get_total_embeddings_cost_df(df_to_embed, column_name_content=column_name_content_ciiu)
            print("Precio estimado DF = $" + str(total_cost) + f" para {total_tokens} tokens")

        documents_embeddings = self.encoder.embed_documents(
            list(df_to_embed[column_name_content_ciiu].values)
        )

        return pd.concat([
            df_to_embed.reset_index(),
            pd.Series(documents_embeddings).to_frame().rename(columns={0: 'embeddings'})
        ], axis=1)
    
    def split_document(self, document_type, path_file, preprocess_function=None):

        documents_text_only = None

        if document_type.lower() == "pdf":
            reader = PyPDF2.PdfReader(path_file)
            print("Pages PDF: ", len(reader.pages))
            pages_text_only = [reader.pages[i].extract_text() for i in range(len(reader.pages))]

            # print(pages_text_only_full[0:100], len(pages_text_only_full))
            documents = self.get_splitter().create_documents([" ".join(pages_text_only)], [{"filename": path_file}])

            if preprocess_function is not None:
                documents_text_only = [preprocess_function(documents[i].page_content) for i in range(len(documents))]
            else:
                documents_text_only = [documents[i].page_content for i in range(len(documents))]
        else:
            raise Exception("Not implemented")
        
        return documents_text_only

    def embed_pdf(self, path_file, check_cost=True, preprocess_function=None):

        if check_cost:
            chunks_pdf = self.split_document("PDF", path_file)

            total_tokens = Embedder.num_tokens_from_string("".join(chunks_pdf))
            total_cost = Embedder.get_embedding_cost(total_tokens)
            print("Precio estimado PDF: $" + str(total_cost) + f" para {total_tokens} tokens")
            
        
        documents_text_only = self.split_document("pdf", path_file, preprocess_function)
        documents_embeddings = self.encoder.embed_documents(
            documents_text_only
        )

        return pd.concat([
            pd.Series(documents_text_only).to_frame().rename(columns={0: 'content'}),
            pd.Series(documents_embeddings).to_frame().rename(columns={0: 'embeddings'})
        ], axis=1)


    @staticmethod
    def num_tokens_from_string(string: str, encoding_name = "cl100k_base") -> int:
        if not string:
            return 0
        encoding = tiktoken.get_encoding(encoding_name)
        num_tokens = len(encoding.encode(string))
        return num_tokens


    @staticmethod
    def get_embedding_cost(num_tokens):
        return num_tokens/1000*0.0001


    @staticmethod
    def get_total_embeddings_cost_df(df, column_name_content='content'):
        total_tokens = 0
        for i in df.index:
            text = df[column_name_content][i]
            token_len = Embedder.num_tokens_from_string(text)
            total_tokens = total_tokens + token_len
        total_cost = Embedder.get_embedding_cost(total_tokens)
        return total_cost, total_tokens
    

import boto3
from langchain.embeddings import BedrockEmbeddings


class EmbedderAWS(Embedder):
    
    def __init__(self, configuration, encoder=None):
        super().__init__(configuration, encoder)

        self.client_bedrock = boto3.client(
            'bedrock-runtime',
            region_name=configuration["provider"]["region"],
            aws_access_key_id=configuration["provider"]["credentials"]["aws_access_key_id"],
            aws_secret_access_key=configuration["provider"]["credentials"]["aws_secret_access_key"]
        )

        if encoder is None:
            self.encoder = BedrockEmbeddings(
                model_id=self.config_model["id"],
                client=self.client_bedrock
            )

In [131]:
configuration = {
    "provider": {
        "region": os.environ.get("aws_region", None),
        "credentials": {
            "aws_access_key_id": os.environ.get("aws_access_key", None),
            "aws_secret_access_key": os.environ.get("aws_secret_key", None)
        }
    },
    "model": {
        "id": "amazon.titan-embed-text-v1"
    },
    "splitter":{
        "chunk_size": 1024,
        "chunk_overlap": 50,
        "length_function": len,
        "add_start_index": True
    }
}

embedder = EmbedderAWS(configuration)

# Usage: Embeddings

In [13]:
# CIIU nivel 6
total_cost, total_tokens = Embedder.get_total_embeddings_cost_df(df_ciiu_4_level6, column_name_content_ciiu=column_name_content_ciiu)
print("Precio estimado para tokens de CIIU nivel 6= $" + str(total_cost) + f" para {total_tokens} tokens")

df_ciiu_4_level6 = embedder.split_and_explode(df_ciiu_4_level6, column_name_content_ciiu)
print(df_ciiu_4_level6.count())

Precio estimado = $0.0076984 para 76984 tokens


In [ ]:
df_ciiu_4_level6 = embedder.embed_dataframe(df_ciiu_4_level6, column_name_content_ciiu, check_cost=True)

print(df_ciiu_4_level6.count())
df_ciiu_4_level6.head(10)

In [40]:
file_name_0 = f"{path_data}/ley_organica_de_proteccion_de_datos_personales.pdf"
chunks_pdf = embedder.split_document("PDF", file_name_0)
print("Chunks: ", len(chunks_pdf))

total_tokens = Embedder.num_tokens_from_string("".join(chunks_pdf))
total_cost = Embedder.get_embedding_cost(total_tokens)
print("Precio estimado para tokens de PDF LODPD= $" + str(total_cost) + f" para {total_tokens} tokens")

Pages PDF:  38
Chunks:  114
Precio estimado para tokens de PDF LODPD= $0.0035543999999999997 para 35544 tokens


In [108]:
def preprocesspdf(text):
    return text.replace("\n", " ")

print(preprocesspdf("Hola\nMúndó!"))

Hola Múndó!


In [132]:
# PDF: ley organica de datos personales
df_pdf_lodpd_embeddings = embedder.embed_pdf(file_name_0, check_cost=True, preprocess_function=preprocesspdf)
print(df_pdf_lodpd_embeddings.count())
df_pdf_lodpd_embeddings.head(10)

Pages PDF:  38
Precio estimado PDF: $0.0035664000000000004 para 35664 tokens
Pages PDF:  38
content       146
embeddings    146
dtype: int64


,content,embeddings
0,LEY ORGÁNICA DE PROTECCIÓN DE DATOS PERSONALES...,"[0.8671875, -0.06982422, 0.28515625, -0.233398..."
1,"Adicionalmente, agradeceré a usted que, una ve...","[1.171875, -0.19824219, 0.18359375, -0.1103515..."
2,"Estado constitucional de derechos y justicia, ...","[0.53515625, -0.17089844, 0.31054688, 0.328125..."
3,"autoridades garantizarán su cumplimiento.""; Qu...","[0.49023438, -0.18847656, 0.12451172, 0.080078..."
4,injustificadamente el ejercicio de los derecho...,"[0.08251953, -0.19433594, 0.24511719, 0.125976..."
5,las personas a lo largo de su vida y un deber ...,"[0.6640625, -0.14550781, 0.33398438, 0.0268554..."
6,desastres naturales o antropogénicos. El Estad...,"[0.6796875, -0.20214844, 0.16601562, 0.2675781..."
7,"personas: ""19. El derecho a la protección de d...","[0.58203125, 0.013793945, 0.34375, -0.32617188..."
8,"misma, o sobre sus bienes, consten en entidade...","[0.9375, -0.07080078, 0.19042969, -0.32226562,..."
9,constituye un servicio a la colectividad que s...,"[0.65234375, -0.123046875, 0.29492188, 0.07470..."


# Postgres - pgvector

## Connection

In [52]:
configuration_db = {
    "postgresql": {
        "database": os.environ.get("postgres_database", None),
        "user": os.environ.get("postgres_user", None),
        "password": os.environ.get("postgres_password", None),
        "host": os.environ.get("postgres_host", None),
        "port": os.environ.get("postgres_port", None),
    }
}

In [159]:
import psycopg2
from pgvector.psycopg2 import register_vector
from psycopg2.extras import execute_values

class PostgresHandler:

    def __init__(self, configuration_db, verbose=1):
        self.configuration_db = configuration_db
        self.conn = None
        self.verbose = verbose

    def log(self, message, minimum_verbose=1):

        if self.verbose >= minimum_verbose:
            message_len = len(message)

            if message_len > 2000:
                print(message[0:2000])
            else:
                print(message)

    def connect(self):

        self.close()
        # Establishing the connection
        self.conn = psycopg2.connect(
            database=self.configuration_db["postgresql"]["database"],
            user=self.configuration_db["postgresql"]["user"],
            password=self.configuration_db["postgresql"]["password"],
            host=self.configuration_db["postgresql"]["host"],
            port=self.configuration_db["postgresql"]["port"]
        )

    def close(self):
        if self.conn is not None:
            self.conn.close()

        self.conn = None

    def __del__(self):
        self.close()

    def _check_open(self, open_and_close):

        if open_and_close:
            self.connect()
        else:
            if self.conn is None:
                raise Exception("Create connection first (use self.connect() and other functions needed for startup)")
            
    def _check_commit_and_close(self, commit=True, open_and_close=False):
        if commit:
            self.conn.commit()

        if open_and_close:
            self.close()

    def execute_statement(self, statement, commit=True, open_and_close=False):
        self._check_open(open_and_close)
        
        cursor = self.conn.cursor()
        cursor.execute(statement)

        self._check_commit_and_close(commit, open_and_close)
        return cursor
        

    def enable_vector_extension(self):
        self.execute_statement("CREATE EXTENSION IF NOT EXISTS vector", open_and_close=True)

    def register_vector_in_conn(self):
        register_vector(self.conn)

    def insert_values(self, data_list, sql_columns, register_vector_conn=True, commit=True, open_and_close=False):
        self._check_open(open_and_close)

        if register_vector_conn:
           self.register_vector_in_conn()

        cursor = self.conn.cursor()

        execute_values(
            cursor, 
            f"INSERT INTO {sql_columns} VALUES %s", 
            data_list
        )

        self._check_commit_and_close(commit, open_and_close)


    def get_similar_docs(self, embedder, user_input, schema, table_name, k=3, custom_where="", columns="content", register_vector_conn=True, commit=True, open_and_close=False):
        """Obtiene el top 3 documentos similares a un query determinado por la distancia definida en el índice
    
            Parameters:
                embedder (Embedder): Objeto instanciado de la clase Embedder con su configuración
                user_input (str): String con la consulta del usuario
                schema (str): Schema de postgres en donde está la tabla con los embeddings
                table_name (str): Nombre de la tabla con los embeddings
                k (int): Número de documentos a recopilar de la base de datos
                register_vector_before (bool): indicador de si debe usarse register_vector (True) o no. Si no se registró al vector antes o se cerró la conexión entonces es necesario ponerlo en True
                custom_where (str): Condición where para prefiltrar datos de la tabla de embeddings
                columns (str): Comma separated strings of columns to retrieve
                register_vector_conn (bool): indicador de si debe usarse register_vector (True) o no. Si no se registró al vector antes o se cerró la conexión entonces es necesario ponerlo en True
                commit (bool): indicador de si se debe hacer un commit en la operación al final o no
                open_and_close (bool): indicador de si debe auto gestionar el inicio y final de la conexión

            Returns:
                top_docs (List[Tuple]): Lista que contiene tantas tuplas como k. Cada uno indicando su distancia al final junto a varios atributos predefinidos
            
        """
        
        self._check_open(open_and_close)

        if register_vector_conn:
           self.register_vector_in_conn()

        self.embedding_array = np.array(embedder.encoder.embed_query(user_input))

        cur = self.conn.cursor()

        # Get the top k most similar documents using the KNN <=> operator
        cur.execute(f"SELECT {columns}, embedding <=> %s as distance FROM {schema}.{table_name} {custom_where} ORDER BY distance LIMIT {k}", (self.embedding_array,))
        
        top_docs = cur.fetchall()
        
        self._check_commit_and_close(commit, open_and_close)

        return top_docs

## Setup

In [160]:
postgres_handler = PostgresHandler(configuration_db, verbose=10)

In [79]:
postgres_handler.enable_vector_extension()

In [ ]:
postgres_handler.connect()
# DROP TABLE IF EXISTS public.pg_embeddings_ciiu
postgres_handler.execute_statement("""CREATE TABLE IF NOT EXISTS public.pg_embeddings_ciiu (
    id int GENERATED BY DEFAULT AS IDENTITY PRIMARY KEY,
    content text,
    embedding vector(1536),
    code VARCHAR(255),
    level VARCHAR(2),
    embedding_tool VARCHAR(255),
    source VARCHAR(255) DEFAULT 'CIIU CATALOG',
    version varchar(3) DEFAULT '1',
    date_update TIMESTAMP DEFAULT NOW()
);
CREATE INDEX IF NOT EXISTS ON public.pg_embeddings_ciiu USING hnsw (embedding vector_cosine_ops) WITH (m = 16, ef_construction = 64);
""")
postgres_handler.close()

In [176]:
postgres_handler.connect()

postgres_handler.execute_statement("""DROP TABLE IF EXISTS public.pg_embeddings_general;
CREATE TABLE IF NOT EXISTS public.pg_embeddings_general (
    id int GENERATED BY DEFAULT AS IDENTITY PRIMARY KEY,
    content text,
    embedding vector(1536),
    embedding_tool VARCHAR(255),
    source VARCHAR(255),
    version varchar(3) DEFAULT '1',
    date_update TIMESTAMP DEFAULT NOW()
);
CREATE INDEX ON public.pg_embeddings_general USING hnsw (embedding vector_cosine_ops) WITH (m = 16, ef_construction = 64);
""")
postgres_handler.close()

## ETL Postgres

In [55]:
data_list_ciiu = [
    (
        row[column_name_content_ciiu], 
        np.array(row['embeddings']), 
        row['CIIU4_COD'], 
        row['CIIU4_NIVEL'], 
        configuration["model"]["id"]
    ) for index, row in df_ciiu_4_level6.iterrows()
]

postgres_handler.insert_values(
    data_list_ciiu,
    sql_columns="public.pg_embeddings_ciiu (content, embedding, code, level, embedding_tool)",
    register_vector_conn=True,
    commit=True,
    open_and_close=True
)

In [178]:
data_list_pdf_0 = [
    (
        row["content"], 
        np.array(row['embeddings']), 
        configuration["model"]["id"],
        file_name_0.split("/")[-1],
        "1"
    ) for index, row in df_pdf_lodpd_embeddings.iterrows()
]

postgres_handler.insert_values(
    data_list_pdf_0,
    sql_columns="public.pg_embeddings_general (content, embedding, embedding_tool, source, version)",
    register_vector_conn=True,
    commit=True,
    open_and_close=True
)

In [179]:
postgres_handler.connect()
print(postgres_handler.execute_statement("""
SELECT content FROM public.pg_embeddings_general WHERE LOWER(content) like '%litardo%'
""", open_and_close=False).fetchall()[0:10])
postgres_handler.close()

[('LEY ORGÁNICA DE PROTECCIÓN DE DATOS PERSONALES Ley 0 Registro Oficial Suplemento 459 de 26-may.-2021 Estado: Vigente ASAMBLEA NACIONAL LEY ORGÁNICA DE PROTECCIÓN DE DATOS PERSONALES PRESIDENCIA DE LA REPÚBLICA DEL ECUADOR Oficio No. T. 680-SGJ-21-0263 Quito, 21 de mayo de 2021 Señor Ingeniero Hugo Del Pozo Barrezueta DIRECTOR DEL REGISTRO OFICIAL En su despacho De mi consideración: Con oficio número PAN-CLC-2021-0384 de 11 de mayo de 2021, el señor Ingeniero César Litardo Caicedo, Presidente de la Asamblea Nacional, remitió el proyecto de LEY ORGÁNICA DE PROTECCIÓN DE DATOS PERSONALES. Dicho proyecto de ley ha sido sancionado por el señor Presidente de la República, el día de hoy, por lo que, conforme a lo dispuesto en los artículos 137 de la Constitución de la República y 63 de la Ley Orgánica de la Función Legislativa, se la remito a usted en original y en copia certificada, junto con el certificado de discusión, para su correspondiente publicación en el Registro Oficial.',), ('Da

# Usage: Getting similar docs

Requirements: instantiate embedder and create connection to postgres

In [72]:
user_input_ciiu = "soldador"
related_docs_ciiu = postgres_handler.get_similar_docs(
    embedder,
    user_input=user_input_ciiu,
    schema="public",
    table_name="pg_embeddings_ciiu",
    k=3,
    custom_where="WHERE level = '6'",  # WHERE level = 2
    columns="code, content",
    register_vector_conn=True,
    commit=True,
    open_and_close=True
)
print(f"Top documentos relacionados para el query: {user_input_ciiu}")
for related in related_docs_ciiu:
    print(related)

Top documentos relacionados para el query: soldador
('C2593.14', 'Fabricación de machetes, espadas, bayonetas, etcétera.', 0.576861512125129)
('C1410.01', 'Fabricación de prendas de vestir de cuero o cuero regenerado, incluidos accesorios de trabajo de cuero como: mandiles para soldadores, ropa de trabajo, etcétera.', 0.6866575668773417)
('G4620.15', 'Venta al por mayor de flores y plantas.', 0.7138840390908127)


In [180]:
postgres_handler.connect()
postgres_handler.register_vector_in_conn()

In [181]:
user_input_pdf = "señor Ingeniero César Litardo"
related_docs_pdf = postgres_handler.get_similar_docs(
    embedder,
    user_input=user_input_pdf,
    schema="public",
    table_name="pg_embeddings_general",
    k=3,
    custom_where="",  # WHERE source = 'ley_organica_de_proteccion_de_datos_personales.pdf'
    columns="version, content, source",
    register_vector_conn=True,
    commit=True,
    open_and_close=False
)
print(f"Top documentos relacionados para el query: {user_input_pdf}")
for i, related in enumerate(related_docs_pdf):
    print(f"Doc {i+1} version:{related[0]} ({related[3]}): {related[1]}\n")


"""
SET hnsw.ef_search = 100;
A higher value provides better recall at the cost of speed.

Use SET LOCAL inside a transaction to set it for a single query

BEGIN;
SET LOCAL hnsw.ef_search = 100;
SELECT ...
COMMIT;

"""

SELECT version, content, source, embedding <=> [-0.17480469 -0.41601562  0.00643921 ...  0.17382812 -0.34179688
 -0.921875  ] as distance FROM public.pg_embeddings_general  ORDER BY distance LIMIT 3
Top documentos relacionados para el query: señor Ingeniero César Litardo
Doc 1 version:1 (0.5661279907125014): Dado y suscrito, a los diez días del mes de mayo de del año dos mil veintiuno. ING. CÉSAR LITARDO CAICEDO Presidente DR. JAVIER RUBIO DUQUE Secretario General PALACIO NACIONAL, DISTRITO METROPOLITANO DE QUITO, A VEINTIUNO DE MAYO DE DOS MIL VEINTIUNO. SANCIÓNASE Y PROMÚLGASE Lenin Moreno Garcés PRESIDENTE CONSTITUCIONAL DE LA REPÚBLICA Es fiel copia del original.-Lo Certifico Quito, 21 de mayo del 2021 Dra. Johana Pesántez Benítez SECRETARIA GENERAL JURÍDICA PRESIDENCIA DE LA REPÚBLICA. LEY ORGÁNICA DE PROTECCIÓN DE DATOS PERSONALES - Página 38 FINDER LOYAL - www.lexis.com.ec

Doc 2 version:1 (0.6468258015937673): FINDER LOYAL - www.lexis.com.ec En el caso que los datos se obtengan

In [167]:
with open("../000_output/output_sql.txt", "w") as f_out:
    sql_save = f"SELECT version, content, source, embedding <=> {list(postgres_handler.embedding_array)} as distance FROM public.pg_embeddings_general ORDER BY distance LIMIT 3"
    f_out.write(sql_save)    

In [171]:
df_result = postgres_handler.execute_statement("""
SELECT content FROM public.pg_embeddings_general WHERE LOWER(content) like '%litardo%'
""").fetchall()
df_result

[('LEY ORGÁNICA DE PROTECCIÓN DE DATOS PERSONALES Ley 0 Registro Oficial Suplemento 459 de 26-may.-2021 Estado: Vigente ASAMBLEA NACIONAL LEY ORGÁNICA DE PROTECCIÓN DE DATOS PERSONALES PRESIDENCIA DE LA REPÚBLICA DEL ECUADOR Oficio No. T. 680-SGJ-21-0263 Quito, 21 de mayo de 2021 Señor Ingeniero Hugo Del Pozo Barrezueta DIRECTOR DEL REGISTRO OFICIAL En su despacho De mi consideración: Con oficio número PAN-CLC-2021-0384 de 11 de mayo de 2021, el señor Ingeniero César Litardo Caicedo, Presidente de la Asamblea Nacional, remitió el proyecto de LEY ORGÁNICA DE PROTECCIÓN DE DATOS PERSONALES. Dicho proyecto de ley ha sido sancionado por el señor Presidente de la República, el día de hoy, por lo que, conforme a lo dispuesto en los artículos 137 de la Constitución de la República y 63 de la Ley Orgánica de la Función Legislativa, se la remito a usted en original y en copia certificada, junto con el certificado de discusión, para su correspondiente publicación en el Registro Oficial.',),
 ('D

# Clean UP

In [108]:
del postgres_handler

# END